# Train Daily Revenue Sproc

## With pandas dfs

In [ ]:

import pandas as pd
import numpy as np
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session

from snowflake.snowpark.functions import sproc
from snowflake.ml.registry import Registry
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from snowflake.snowpark.context import get_active_session
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

session = get_active_session()


sproc_name = 'retail_demo.Silver.Daily_Revenue_Train_XGB'
stage_name = 'retail_demo.Silver.Model_stage'

@sproc(name=sproc_name,  
       stage_location=stage_name,  
       is_permanent=True, 
       replace=True, 
       packages=[
        "snowflake-snowpark-python",
        'snowflake-ml-python', 
        'xgboost',
        'pandas', 
         ])
def train_and_save_model(session: Session, major_version: bool = True) -> str:
    model_name = 'DAILY_REVENUE_XGB'
    train_vw_name =  'retail_demo.silver.vw_daily_revenue_train'
    test_vw_name  =  'retail_demo.silver.vw_daily_revenue_test'
    
    target_cols = ['TOTAL_AMOUNT']  

    train_df = session.table(train_vw_name)
    test_df = session.table(test_vw_name)
    
    ts_features = session.table('Retail_demo.Silver.Date_features')
    train_df = train_df.join(ts_features, train_df['TRANSACTION_DATE'] == ts_features["DATE_"])
    results_df = test_df.drop( "TOTAL_AMOUNT").to_pandas()
                
    X_train = train_df.drop("Transaction_date", "date_", "TOTAL_AMOUNT").to_pandas()
    y_train = train_df[["TOTAL_AMOUNT"]].to_pandas()
    test_df = test_df.join(ts_features, test_df['TRANSACTION_DATE'] == ts_features["DATE_"])
                
    X_test = test_df.drop("Transaction_date", "date_").to_pandas()
    y_test = X_test["TOTAL_AMOUNT"]
    X_test = X_test.drop(columns = ["TOTAL_AMOUNT"])
               
    param_grid = {
        'learning_rate': [ 0.05, 0.1],
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
                    
                    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
    tscv = TimeSeriesSplit(n_splits=5)
    grid_search = GridSearchCV(estimator=xgb_model, 
                               param_grid=param_grid, 
                               scoring='neg_mean_squared_error', 
                               cv=tscv, 
                              )


    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
 
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    results_df['Predicted_Amount'] = y_pred
    
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    #r2 = r2_score(y_test, y_pred)
    #msle = np.mean(np.square(np.log1p(y_test) - np.log1p(y_pred)))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mse = mean_squared_error(y_test, y_pred)
    sf_df = session.write_pandas(results_df,database = "RETAIL_DEMO", schema = "GOLD", table_name = "XGB_FORECASTED_DAILY_REVENUE", auto_create_table=True, overwrite=True)
    def set_model_version(registry_object,model_name, major_version):
        import numpy as np
        import json
        
        model_list = registry_object.show_models()
        if len(model_list) == 0:
            return 'V1'
        
        model_list_filter = model_list[model_list['name'] ==  model_name]
        if len(model_list_filter) == 0:
            return 'V1'

        version_list_string = model_list_filter['versions'].iloc[0]
        version_list = json.loads(version_list_string)
        version_numbers = [float(s.replace('V', '')) for s in version_list]
        model_last_version = max(version_numbers)
        
        if np.isnan(model_last_version) == True:
            model_new_version = 'V1'

        elif np.isnan(model_last_version) == False and major_version == True:
            model_new_version = round(model_last_version + 1,2)
            model_new_version = 'V' + str(model_new_version)
            
        else:
            model_new_version = round(model_last_version + .1,2)
            model_new_version = 'V' + str(model_new_version)
          
        return model_new_version # This is the version we will use when we log the new model.
    # Create model regisry object
    try:
        model_registry = Registry(session=session, database_name=session.get_current_database(), schema_name='SILVER')
    except Exception as e:
        return (f'Error with creating model registry object: {e}')

    try:
        version_name = set_model_version(model_registry, model_name, major_version=major_version)
            
        X = X_train.head(100)
        model_version = model_registry.log_model(
                        best_model, 
                        model_name = model_name, 
                        version_name= f'"{version_name}"',
                        sample_input_data=X,
                        conda_dependencies=['snowflake-snowpark-python','snowflake-ml-python','scikit-learn', 'pandas', 'numpy', 'xgboost'],
                        options={'relax_version': True},
                        )
        model_version.set_metric(metric_name='mean_abs_pct_err', value=mape)
        model_version.set_metric(metric_name='mean_sq_err', value=mse)
                
    except Exception as e:
        print(f'Error with saving model to registry: {e}')
                
    try:
        session.sql(f'alter model {model_name} set default_version = "{version_name}";')
    except Exception as e:
        print( f'Error with setting default version: {e}')
    return f'Model {model_name} has been logged with version {version_name} and has a MAPE of {mape} and MSE of {rmse}'  


In [ ]:
CALL retail_demo.silver.Daily_Revenue_Train_XGB(False);

# Predict and write to gold Sproc


In [ ]:

import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session

from snowflake.snowpark.functions import sproc
from snowflake.ml.registry import Registry
from snowflake.snowpark.context import get_active_session
session = get_active_session()


sproc_name = 'retail_demo.Silver.Daily_Rev_Pred_Write'
stage_name = 'retail_demo.Silver.Model_stage'

@sproc(name=sproc_name,  
       stage_location=stage_name,  
       is_permanent=True, 
       replace=True, 
       packages=[
        "snowflake-snowpark-python",
        'snowflake-ml-python', 
        'pandas', 
         ])
def predict_and_write(session: Session) -> str:
    sf_df = session.sql("""
            select * from (
            WITH Last_Date_CTE AS (
                SELECT MAX(transaction_date) AS Last_Date
                FROM retail_demo.silver.daily_revenue
            ),
            Future_Dates_CTE AS (
                SELECT DATEADD(DAY, SEQ4(), Last_Date) AS Transaction_date
                FROM Last_Date_CTE,
                     TABLE(GENERATOR(ROWCOUNT => 120))
            )
            SELECT 
                Transaction_date,
            FROM Future_Dates_CTE
            
            order by Transaction_date)
            """)
    results_df = sf_df.to_pandas()
    ts_features = session.table('Retail_demo.Silver.Date_features')
    sf_df = sf_df.join(ts_features, sf_df['TRANSACTION_DATE'] == ts_features["DATE_"])
    try: 
        model_registry = Registry(session=session, database_name=session.get_current_database(), schema_name='SILVER')
        model_version = model_registry.get_model('DAILY_REVENUE_XGB').default    
        preds = model_version.run(sf_df,function_name = 'predict')
        results_df["Predicted_Amount"]  = preds.select('"output_feature_0"').to_pandas()
    except Exception as e:
        print( f'Error with making predictions: {e}')
    
    if not results_df.empty:
        try:
            # Attempt to write the DataFrame to Snowflake
            temp_df = session.write_pandas(results_df,database = "RETAIL_DEMO", 
                                                              schema = "GOLD", 
                                                              table_name = "XGB_FORECASTED_DAILY_REVENUE", 
                                                              uto_create_table = False,
                                                              overwrite=False)
            return  "Successfully predicted and written to table"
        except Exception as e:
            return  (f"An error occurred: {e}")
    else:
        return "Dataframe is empty"

In [ ]:
CALL retail_demo.silver.Daily_Rev_Pred_Write();